# Steady state solution

In [57]:
using gmsh

using LinearAlgebra, SparseArrays
using WriteVTK

using BenchmarkTools

include("../constants.jl")
include("../get_mesh_data.jl")
include("../process.jl")
include("../definitions/source.jl")
include("../definitions/linear_reluctivity.jl")
include("../definitions/assemble_Kf.jl")

const MESH_LOCATION = "../../mesh/transformer_stedin.msh"
const OUTPUT_LOCATION = "../../out/"

using Logging

In [58]:
gmsh.open(MESH_LOCATION)
mshdata = get_mesh_data();

Info    : Reading '../../mesh/transformer_stedin.msh'...
Info    : 168 entities
Info    : 10385 nodes
Info    : 20768 elements
Info    : Done reading '../../mesh/transformer_stedin.msh'


## Problem definition

Find $A_z$ in the system
$$ -\nabla \times \left[\frac{1}{\mu}\nabla A_z\right] = \mathbf J_0, $$
where
- $A_z$ is the current density in the $z$ direction.
- $\mu$ is the permeability of the core.
- $\mathbf J_0$ is the imposed source current density.
- $\sigma$: the conductivity of the material (not used).

### Parameters

In [59]:
"Primary peak phase current"
Ip = 0;

"Secondary peak phase current"
Is = 777.62;

Np = 266;
Ns = 6;

# Calculate current density in the windings
Jp = Np * Ip / Awhv;
Js = Ns * Is / Awlv;

# Vacuum permeability
mu0 = 4e-7 * pi;

# Relative permeability of the core
mur = 1000;    

### Source, linear reluctivity per element (no conductivity due to steady state)


In [60]:
group_id_per_element = mshdata.e_group

source_func(group_id) = Jp * exp(1im * 2pi/3) * (-1 * (group_id==3) + 1 * (group_id==4)) + 
                           Jp * (-1 * (group_id==5) + 1 * (group_id==6)) + 
                           Jp * exp(-1im * 2pi/3) * (-1 * (group_id==7) + 1 * (group_id==8)) + 
                           Js * exp(1im * 2pi/3) * (1 * (group_id==9) - 1 * (group_id==10)) +
                           Js * (1 * (group_id==11) - 1 * (group_id==12)) + 
                           Js * exp(-1im * 2pi/3) * (1 * (group_id==13) - 1 * (group_id==14));
source_per_element = map(source_func, group_id_per_element);

# Relative permeability model
reluctivity_per_element = map(linear_reluctivity, group_id_per_element);

zero_conductivity(group_id) = 0
conductivity_per_element = map(zero_conductivity, group_id_per_element);

## System of equations

In [64]:

# with_logger(ConsoleLogger(stderr, Logging.Debug)) do
K, f = assemble_Kf(mshdata, source_per_element, reluctivity_per_element);
# end


(sparse([1, 73, 292, 3543, 2, 123, 124, 3544, 3, 182  …  2470, 10311, 10382, 10384, 72, 2543, 2544, 10310, 10383, 10385], [1, 1, 1, 1, 2, 2, 2, 2, 3, 3  …  10384, 10384, 10384, 10384, 10385, 10385, 10385, 10385, 10385, 10385], ComplexF64[1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, -296302.7798814594 + 0.0im, 1.0 + 0.0im, 0.0 + 0.0im, 0.0 + 0.0im, -296302.7798870144 + 0.0im, 1.0 + 0.0im, 0.0 + 0.0im  …  -831566.3069097897 + 0.0im, -313835.0144328627 + 0.0im, -677428.4886853056 + 0.0im, 3.0142441240861635e6 + 0.0im, -578864.0635942314 + 0.0im, -612550.2504638598 + 0.0im, -831566.3069096752 + 0.0im, -313835.01443294587 + 0.0im, -677428.4886854272 + 0.0im, 3.0142441240861393e6 + 0.0im], 10385, 10385), ComplexF64[0.0 + 0.0im; 0.0 + 0.0im; … ; 7.496634577631168 + 12.98455197423484im; 7.4966345776312595 + 12.984551974234996im;;])

How long does this take?

In [65]:
# @timev K, f = assemble_Kf(mshdata, source_per_element, reluctivity_per_element);

Solve the system.

In [66]:
u = K \ f;

## Post

In [67]:
# Post-process for magnetic field and current density
omega = 0
B, H, Wm, Jel = process(mshdata, u, source_per_element, reluctivity_per_element, conductivity_per_element, omega);
Bnorm = norm.(sqrt.(B[1].^2 + B[2].^2));

Save the result

In [68]:
# Define nodes (points) and elements (cells)
points = [mshdata.xnode mshdata.ynode]';
cells = [MeshCell(VTKCellTypes.VTK_TRIANGLE, el) for el in mshdata.elements];

# Create VTK file structure using nodes and elements
vtkfile = vtk_grid(string(OUTPUT_LOCATION, "transformer7.vtu"), points, cells);

# Store data in the VTK file
vtkfile["Az", VTKPointData()]   = norm.(u);
vtkfile["imA", VTKPointData()]  = imag.(u);
vtkfile["Bnorm", VTKCellData()] = Bnorm;
vtkfile["Jel", VTKCellData()]   = Jel;

# Save the file
print("Saving result in a file...")
outfiles = vtk_save(vtkfile);
println(" Done.")

Saving result in a file... Done.
